In [0]:
df = spark.read.table("bronze_layer")
# Ver as primeiras 5 linhas
# print(df.head())

# Número de linhas
print(df.count())

df.explain(mode="cost")


# Ver informações sobre tipos de dados e valores nulos
# Isso também mostra o uso de memória do DataFrame no Python
# print(df.info()) -- não funciona

# Estatísticas descritivas (média, min, max, etc. para colunas numéricas)
# print(df.describe())

# Verificar se existem valores faltantes por coluna
# print(df.isnull().sum()) -- não funciona

# Ver os nomes de todas as colunas
# print(df.columns.tolist())

In [0]:
# 2. Descrição do arquivo

# Tamanho do arquivo em Bytes (disco)
file_size_bytes = os.path.getsize()
file_size_mb = file_size_bytes / (1024 * 1024)

# Carregar os dados (ajuste o encoding/sep se necessário)
df = pd.read_csv(file_path, sep=',', encoding='MacRoman')

# Obter Linhas e Colunas
linhas, colunas = df.shape

print(f"--- Estatísticas do Arquivo ---")
print(f"Tamanho em disco: {file_size_bytes} bytes ({file_size_mb:.2f} MB)")
print(f"Total de Linhas: {linhas}")
print(f"Total de Colunas: {colunas}")

In [0]:
from pyspark.sql import functions as F

# 1. Carregar o DataFrame (se estiver no pipeline DLT use dlt.read, senão use spark.table)
df = spark.table("stg_sus_establishments") 

# 2. Criar uma expressão que conta apenas valores que NÃO são nulos nem vazios
# O Spark percorre a tabela uma única vez para todas as colunas
null_counts = df.select([
    F.count(
        F.when(
            (F.col(c).isNotNull()) & 
            (F.col(c) != "") & 
            (F.trim(F.col(c)) != ""), 
            c
        )
    ).alias(c)
    for c in df.columns
]).collect()[0].asDict()

print(null_counts)

# 3. Filtrar apenas as colunas onde a contagem de valores válidos é 0
colunas_vazias = [col for col, count in null_counts.items() if count == 0]

# --- Resultado ---
if colunas_vazias:
    print(f"⚠️ Encontradas {len(colunas_vazias)} colunas totalmente vazias:")
    print(colunas_vazias)
else:
    print("✅ Nenhuma coluna está totalmente vazia.")